In [1]:
import copy
import numpy as np
from tqdm import tqdm
import warnings
from numpy.random import normal, randint
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
import re
%load_ext autoreload
%autoreload 2

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


from inn import *
from datasetall.datautils import *
from clfs.clfutils import *

# load dataset utils
dataset = InnDataSet("compas", test_size=0.3)
X, y, df, columns, ordinal_features, discrete_features, continuous_features, feat_var_map = load_dataset_utils(
    dataset)
dataset_name = "compas"

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [2]:
print(X.shape, len(discrete_features))

(6172, 7) 3


In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
# these X test is for testing multiplicity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
seed = 0
clf0 = MLPClassifier(hidden_layer_sizes=(20, 20), random_state=seed, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)
clf0.fit(X_train, y_train)

MLPClassifier(batch_size=64, hidden_layer_sizes=(20, 20),
              learning_rate='adaptive', learning_rate_init=0.02,
              random_state=0)

In [3]:
clf = MLPClassifier(hidden_layer_sizes=(15, 15), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.8440 test_accuracy with a std of 0.0133
0.6847 test_f1_macro with a std of 0.0339


In [4]:
clf = MLPClassifier(hidden_layer_sizes=(15, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.8406 test_accuracy with a std of 0.0127
0.6836 test_f1_macro with a std of 0.0331


In [5]:
clf = MLPClassifier(hidden_layer_sizes=(20, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.8448 test_accuracy with a std of 0.0106
0.6814 test_f1_macro with a std of 0.0215


In [6]:
clf = MLPClassifier(hidden_layer_sizes=(30, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.8440 test_accuracy with a std of 0.0114
0.6875 test_f1_macro with a std of 0.0355


In [7]:
clf = MLPClassifier(hidden_layer_sizes=(30, 25), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.8448 test_accuracy with a std of 0.0115
0.7054 test_f1_macro with a std of 0.0281


In [8]:
clfs = []
total_num = 150
seed = 0
scores_acc = []
scores_compl = []
sizes = [(15,15),(15,20),(20,20),(30,20),(30,25)]
sizes_score = [0, 0.25, 0.5, 0.75, 1]
for i in tqdm(range(0, total_num)):
    X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_train, y_train, test_size=0.2, random_state=2*seed+15)
    sizes_idx = np.random.randint(0,5)
    this_clf = MLPClassifier(hidden_layer_sizes=sizes[sizes_idx], random_state=3*seed+21, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02).fit(X_train_0, y_train_0)
    seed += 12
    if len(np.unique(this_clf.predict(X_test))) == 1:
        continue
    #print(accuracy_score(y_test, this_clf.predict(X_test)))
    scores_acc.append(accuracy_score(this_clf.predict(X_test), y_test))
    scores_compl.append(sizes_score[sizes_idx])
    clfs.append(this_clf)
#clfs.append(clf0)
print(len(clfs))

100%|██████████| 150/150 [02:27<00:00,  1.02it/s]

150


In [10]:
from methods import Datasets, ExperimentsForOneModelSize
d_idxs = np.arange(0, len(X_test))
np.random.seed(1576)
if len(X_test) >= 500:
    d_idxs = np.random.randint(0, len(X_test) - 1, 500)
X_test_run = pd.DataFrame(data=X_test.values[d_idxs], columns=X_test.columns)
y_test_run = pd.DataFrame(data=y_test.values[d_idxs], columns=y_test.columns)
for i, m in enumerate(clfs):
    scores_acc[i] = accuracy_score(y_test_run, m.predict(X_test_run))
d = Datasets(X_train, y_train, X_test_run, y_test_run)

In [11]:
exp_10 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=10, dataset_name="compas", seed=1123)
exp_10.eval()

100%|██████████| 5/5 [01:39<00:00, 19.85s/it]


In [12]:
exp_10.eval_ours()
exp_10.print_results()

100%|██████████| 5/5 [00:08<00:00,  1.62s/it]

model size = 10, mean, single model acc: 0.8562799999999999
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.858 |    0.464 |   0.980 |   0.980 |   0.572 |     0.000 |       0.005 |                    |              |
| mvcva   | 0.858 |    0.464 |   0.980 |   0.174 |   1.000 |     0.505 |       0.005 |                    |              |
| ours    | 0.859 |    0.463 |   0.369 |   0.369 |   1.000 |     0.000 |       0.256 |              0.248 |        0.994 |
| ours-a  | 0.864 |    0.430 |   0.295 |   0.295 |   1.000 |     0.000 |       0.084 |              0.083 |        0.988 |
| ours-s  | 0.860 |    0.657 |   0.306 |   0.306 |   1.000 |     0.000 |       

In [13]:
exp_20 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=20, dataset_name="compas", seed=1123)
exp_20.eval()

100%|██████████| 5/5 [03:36<00:00, 43.38s/it]


In [14]:
exp_20.eval_ours()
exp_20.print_results()

100%|██████████| 5/5 [00:15<00:00,  3.05s/it]

model size = 20, mean, single model acc: 0.85542
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.859 |    0.538 |   0.978 |   0.978 |   0.544 |     0.000 |       0.002 |                    |              |
| mvcva   | 0.859 |    0.538 |   0.978 |   0.111 |   1.000 |     0.603 |       0.002 |                    |              |
| ours    | 0.859 |    0.514 |   0.331 |   0.331 |   1.000 |     0.000 |       0.178 |              0.174 |        0.992 |
| ours-a  | 0.866 |    0.541 |   0.235 |   0.235 |   1.000 |     0.000 |       0.027 |              0.027 |        0.982 |
| ours-s  | 0.857 |    0.609 |   0.304 |   0.304 |   1.000 |     0.000 |       0.136 |    

In [15]:
exp_30 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=30, dataset_name="compas", seed=1123)
exp_30.eval()

100%|██████████| 5/5 [05:51<00:00, 70.24s/it]


In [16]:
exp_30.eval_ours()
exp_30.print_results()

100%|██████████| 5/5 [00:23<00:00,  4.67s/it]

model size = 30, mean, single model acc: 0.8549866666666668
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.859 |    0.527 |   0.976 |   0.976 |   0.530 |     0.000 |       0.002 |                    |              |
| mvcva   | 0.859 |    0.527 |   0.976 |   0.087 |   1.000 |     0.564 |       0.002 |                    |              |
| ours    | 0.862 |    0.519 |   0.308 |   0.308 |   1.000 |     0.000 |       0.133 |              0.130 |        0.990 |
| ours-a  | 0.865 |    0.532 |   0.225 |   0.225 |   1.000 |     0.000 |       0.035 |              0.035 |        0.983 |
| ours-s  | 0.861 |    0.567 |   0.294 |   0.294 |   1.000 |     0.000 |       